In [ ]:
# Install required packages.
import os
import torch
os.environ['TORCH'] = torch.__version__
print(torch.__version__)

!pip install -q torch-scatter -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q torch-sparse -f https://data.pyg.org/whl/torch-${TORCH}.html
!pip install -q git+https://github.com/pyg-team/pytorch_geometric.git

# Helper function for visualization.
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt


def visualize_graph(G, color):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    nx.draw_networkx(G, pos=nx.spring_layout(G, seed=42), with_labels=False,
                     node_color=color, cmap="Set2")
    plt.show()


def visualize_embedding(h, color, epoch=None, loss=None):
    plt.figure(figsize=(7,7))
    plt.xticks([])
    plt.yticks([])
    h = h.detach().cpu().numpy()
    plt.scatter(h[:, 0], h[:, 1], s=140, c=color, cmap="Set2")
    if epoch is not None and loss is not None:
        plt.xlabel(f'Epoch: {epoch}, Loss: {loss.item():.4f}', fontsize=16)
    plt.show()

import os.path as osp

import torch
import torch.nn.functional as F
from sklearn.metrics import roc_auc_score

from torch_geometric.utils import negative_sampling
from torch_geometric.datasets import Planetoid
import torch_geometric.transforms as T
from torch_geometric.nn import GCNConv
from torch_geometric.utils import train_test_split_edges
from torch_geometric.nn import BatchNorm, PNAConv, global_add_pool
from torch_geometric.utils import degree

2.1.0+cu121
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.8/10.8 MB 20.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 5.0/5.0 MB 13.6 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done


In [ ]:
from torch_geometric.datasets import QM9
from torch_geometric.datasets import TUDataset
#dataset = TUDataset('data/ZINC', name='ZINC')

dataset = QM9(root = './data')
print(f'Dataset: {dataset}:')
print('======================')
print(f'Number of graphs: {len(dataset)}')
print(f'Number of features: {dataset.num_features}')
print(f'Number of classes: {dataset.num_classes}')

Extracting data/raw/qm9_v3.zip
Processing...
Using a pre-processed version of the dataset. Please install 'rdkit' to alternatively process the raw data.


Dataset: QM9(130831):
Number of graphs: 130831
Number of features: 11
Number of classes: 19


Done!


In [ ]:
import os.path as osp

import torch
import torch.nn.functional as F
from torch.nn import Embedding, Linear, ModuleList, ReLU, Sequential
from torch.optim.lr_scheduler import ReduceLROnPlateau

from torch_geometric.datasets import ZINC
from torch_geometric.loader import DataLoader
from torch_geometric.nn import BatchNorm, PNAConv, global_add_pool
from torch_geometric.utils import degree


In [ ]:

# Compute the maximum in-degree in the training data.
max_degree = -1
for data in dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    max_degree = max(max_degree, int(d.max()))

# Compute the in-degree histogram tensor
deg = torch.zeros(max_degree + 1, dtype=torch.long)
for data in dataset:
    d = degree(data.edge_index[1], num_nodes=data.num_nodes, dtype=torch.long)
    deg += torch.bincount(d, minlength=deg.numel())



In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F
from torch_geometric.data import Data, DataLoader

from torch.nn import Linear
from torch_geometric.nn import PNAConv
# Initialize the autoencoder model









class GraphAutoencoder(nn.Module):
   def __init__(self):
        super(GraphAutoencoder, self).__init__()
        num_features = 1
        dim = 2


        aggregators = ['mean', 'min', 'max', 'std']
        scalers = ['identity', 'amplification', 'attenuation']


        self.conv1 = PNAConv(in_channels=num_features, out_channels=4,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.bn1 = torch.nn.BatchNorm1d(4)


        self.conv2 = PNAConv(in_channels=4, out_channels=8,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.bn2 = torch.nn.BatchNorm1d(8)


        self.conv3 = PNAConv(in_channels=8, out_channels=4,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.bn3 = torch.nn.BatchNorm1d(4)


        self.conv4 = PNAConv(in_channels=4, out_channels=2,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.bn4 = torch.nn.BatchNorm1d(2)

        self.dconv1 = PNAConv(in_channels=2, out_channels=4,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.dbn1 = torch.nn.BatchNorm1d(4)


        self.dconv2 = PNAConv(in_channels=4, out_channels=8,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.dbn2 = torch.nn.BatchNorm1d(8)


        self.dconv3 = PNAConv(in_channels=8, out_channels=4,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.dbn3 = torch.nn.BatchNorm1d(4)


        self.dconv4 = PNAConv(in_channels=4, out_channels=num_features,
                           aggregators=aggregators, scalers=scalers, deg=deg)
        self.dbn4 = torch.nn.BatchNorm1d(num_features)

   def encode(self,x,edge_index):
        x = F.relu(self.conv1(x, edge_index))
        x = self.bn1(x)
        x = F.relu(self.conv2(x, edge_index))
        x = self.bn2(x)
        x = F.relu(self.conv3(x, edge_index))
        x = self.bn3(x)
        x = F.relu(self.conv4(x, edge_index))
        x = self.bn4(x)
        return x

   def decode(self, z, edge_index): # only pos and neg edges
        z = F.relu(self.dconv1(z, edge_index))
        z = self.dbn1(z)
        z = F.relu(self.dconv2(z, edge_index))
        z = self.dbn2(z)
        z = F.relu(self.dconv3(z, edge_index))
        z = self.dbn3(z)
        z = F.relu(self.dconv4(z, edge_index))
        z = self.dbn4(z)
        return z




# Create a custom graph dataset with 5 graphs, each having 10 nodes and 8 features
#dataset = CustomGraphDataset(num_graphs=5, num_nodes=10, num_features=8)
#num_features = dataset.num_features

epochs = 10
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = "cpu"

model= GraphAutoencoder()
optimizer = torch.optim.Adam(params=model.parameters(), lr=0.001)

criterion = criterion = nn.MSELoss()

batch_size = 1
dataloader = DataLoader(dataset, batch_size=batch_size, shuffle=True)


/usr/local/lib/python3.10/dist-packages/torch_geometric/deprecation.py:26: UserWarning: 'data.DataLoader' is deprecated, use 'loader.DataLoader' instead
  warnings.warn(out)


In [ ]:
from sys import path
import pickle

path=  '/content/modelv28.pkl'
with open(path, 'rb') as file:
      loaded_object = pickle.load(file)

In [ ]:

def linear_beta_schedule(timesteps):
    beta_start = 0.0001
    beta_end = 0.02
    return torch.linspace(beta_start, beta_end, timesteps)

In [ ]:
timesteps = 300

# define beta schedule
betas = linear_beta_schedule(timesteps=timesteps)


# define alphas
alphas = 1. - betas

alphas_cumprod = torch.cumprod(alphas, axis=0)
alphas_cumprod_prev = F.pad(alphas_cumprod[:-1], (1, 0), value=1.0)
sqrt_recip_alphas = torch.sqrt(1.0 / alphas)

# calculations for diffusion q(x_t | x_{t-1}) and others
sqrt_alphas_cumprod = torch.sqrt(alphas_cumprod)
sqrt_one_minus_alphas_cumprod = torch.sqrt(1. - alphas_cumprod)

# calculations for posterior q(x_{t-1} | x_t, x_0)
posterior_variance = betas * (1. - alphas_cumprod_prev) / (1. - alphas_cumprod)

def extract(a, t, x_shape):
    batch_size = t.shape[0]
    out = a.gather(-1, t.cpu())
    return out.reshape(batch_size, *((1,) * (len(x_shape) - 1))).to(t.device)

In [ ]:
# forward diffusion
def q_sample(x_start, t, noise=None):
    if noise is None:
        noise = torch.randn_like(x_start)

    sqrt_alphas_cumprod_t = extract(sqrt_alphas_cumprod, t, x_start.shape)
    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, x_start.shape
    )

    return sqrt_alphas_cumprod_t * x_start + sqrt_one_minus_alphas_cumprod_t * noise, noise

In [ ]:
def get_noisy_image(x_start, t):
  # add noise
  x_noisy,tar_noise = q_sample(x_start, t=t)


  # turn back into PIL image


  return x_noisy,tar_noise

import this for tqdm

In [ ]:
import math
import operator
from itertools import chain, product
from functools import partial
from pathlib import Path
from typing import Any, Optional, Callable, Tuple, Dict, Sequence, NamedTuple

import numpy as np

from tqdm import tqdm

import torch
import torch.nn as nn
from torch import Tensor, LongTensor

In [ ]:
import torch_geometric
from torch_geometric.transforms import BaseTransform, Compose
from torch_geometric.datasets import QM9
from torch_geometric.data import Data
from torch_geometric.loader import DataLoader
from torch_geometric.data import Dataset
from torch_geometric.nn.aggr import SumAggregation
import torch_geometric.nn as geom_nn

import matplotlib as mpl
import matplotlib.pyplot as plt
from torch_scatter import scatter

In [ ]:
@torch.no_grad()
def p_sample(model, data, t, t_index):
    t1 = t_index
    h_time = torch.empty_like(data.x[:, 0:1]).fill_(t1)
    # Concatenate h and h_time along dimension 1
    original_tensor = torch.cat([data.x, h_time], dim=1)


# Reshape to the shape of torch.randn(1, 4, 11)
    reshaped_tensor = original_tensor.view(1, data.num_nodes, 6)  # or use reshape

    original_pos = data.pos
    reshaped_pos = original_pos.view(1, data.num_nodes, 3)
    #reshaped_pos=torch.randn(1, data.num_nodes, 3)
    z,d= diff_model1(reshaped_tensor, reshaped_pos)

    new_z = z[:, :, :5]
    new_z = new_z.squeeze(0)


    betas_t = extract(betas, t, data.x.shape)

    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, data.x.shape
    )
    sqrt_recip_alphas_t = extract(sqrt_recip_alphas, t, data.x.shape)


    # Equation 11 in the paper
    # Use our model (noise predictor) to predict the mean

    model_mean = sqrt_recip_alphas_t * (
        data.x - betas_t * new_z / sqrt_one_minus_alphas_cumprod_t
    )

    if t_index == 0:
        return model_mean
    else:
        posterior_variance_t = extract(posterior_variance, t, data.x.shape)
        noise = torch.randn_like(data.x)
        # Algorithm 2 line 4:
        return model_mean + torch.sqrt(posterior_variance_t) * noise

# Algorithm 2 but save all images:
@torch.no_grad()
def p_sample_loop(model,data):
    device = next(model.parameters()).device

    b = 1
    # start from pure noise (for each example in the batch)
    data2.x = torch.cat([data2.x, data2.pos], dim=1)
    t_init= torch.tensor([299])
    imgs = []
    features,tar= get_noisy_image(data.x, t_init)
    data.x= features
    for i in tqdm(reversed(range(0, timesteps)), desc='sampling loop time step', total=timesteps):


        data.x = p_sample(model, data, torch.full((b,), i, device=device, dtype=torch.long), i)
        imgs.append(data.x.cpu().numpy())
        data.pos=data.x[:, -3:]
    return imgs

@torch.no_grad()
def sample(model, data):
    return p_sample_loop(model, data)

In [ ]:
#come here
@torch.no_grad()
def p_sample(model, data, t, t_index):
    t1 = t_index
    h_time = torch.empty_like(data.x[:, 0:1]).fill_(t1)
    # Concatenate h and h_time along dimension 1
    original_tensor = torch.cat([data.x, h_time], dim=1)


# Reshape to the shape of torch.randn(1, 4, 11)
    reshaped_tensor = original_tensor.view(1, data.num_nodes, 3)  # or use reshape


    original_pos = data.pos
    reshaped_pos = original_pos.view(1, data.num_nodes, 3)
    #reshaped_pos=torch.randn(1, data.num_nodes, 3)
    z,d= diff_model1(reshaped_tensor, reshaped_pos)

    new_z = z[:, :, :2]
    new_z = new_z.squeeze(0)


    betas_t = extract(betas, t, data.x.shape)

    sqrt_one_minus_alphas_cumprod_t = extract(
        sqrt_one_minus_alphas_cumprod, t, data.x.shape
    )
    sqrt_recip_alphas_t = extract(sqrt_recip_alphas, t, data.x.shape)


    # Equation 11 in the paper
    # Use our model (noise predictor) to predict the mean

    model_mean = sqrt_recip_alphas_t * (
        data.x - betas_t * new_z / sqrt_one_minus_alphas_cumprod_t
    )

    if t_index == 0:
        return model_mean
    else:
        posterior_variance_t = extract(posterior_variance, t, data.x.shape)
        noise = torch.randn_like(data.x)
        # Algorithm 2 line 4:
        return model_mean + torch.sqrt(posterior_variance_t) * noise

# Algorithm 2 but save all images:
@torch.no_grad()
def p_sample_loop(model,data):
    device = next(model.parameters()).device

    b = 1
    # start from pure noise (for each example in the batch)
    #data2.x = torch.cat([data2.x, data2.pos], dim=1)
    t_init= torch.tensor([299])
    imgs = []
    features,tar= get_noisy_image(data.x, t_init)
    data.x= features
    #data.pos= features

    for i in tqdm(reversed(range(0, timesteps)), desc='sampling loop time step', total=timesteps):


        data.x = p_sample(model, data, torch.full((b,), i, device=device, dtype=torch.long), i)
        #data.pos= data.x
        imgs.append(data.x.cpu().numpy())
        #data.pos=data.x[:, -3:]
    return imgs

@torch.no_grad()
def sample(model, data):
    return p_sample_loop(model, data)

In [ ]:
from sys import path
import pickle

path=  '/content/modelcoord.pkl'
with open(path, 'rb') as file:
      coord_model = pickle.load(file)

In [ ]:
from sys import path
import pickle

path=  '/content/modelv241.pkl'
with open(path, 'rb') as file:
      diff_model1 = pickle.load(file)